# Classificação de multi-etiquetas de vídeo

Este notebook corresponde a atividade da seção 3.6 no livro do curso. Esta atividade mostra de forma simples, como implementar um programa para realizar classificação de multi-etiquetas de vídeo. Para isso, é implementado um modelo de rede neural que agregar features de diferentes modalidades para realizar multiclassificação.

Na pasta "dataset" existe uma coleção de tabelas csv que contém features (embeddings) audio-visuais de 50 mil vídeos que foram extraídos de uma parte do dataset Youtube8M. Os embeddings visuais foram extraídas da última camada da rede Inception-Resnet-v2 pré-treinada com o dataset ImageNet. Já as features de áudio foram extraídas da última camada da rede Audio-VGG pré-treinada com o dataset AudioSet. A imagem abaixo ilustra o todo o processo de classificação, nesta atividade apenas o processo de agregação de features é implementado.  


<img src="images/multi_modal_arq.png" width="700px" />


## 1 - Pacotes

Execute o bloco abaixo para importar os pacotes necessarios. 

- [tensorflow](https://www.tensorflow.org/) um framework para deep learning
- [numpy](www.numpy.org) pacote de bilbiotecas para computação científica.

In [1]:
import tensorflow as tf
import numpy as np

#códigos locais com funções necessárias para a atividade
from read_batch import *
from evaluate import *

/usr/local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## 2 - Modelo para Multiclassificação

Nesta etapa é definido um modelo para multiclassificação. No código abaixo, a função "build_deep_fully_network" recebe como parâmetro a quantidade de features e número de classes do problema e cria uma uma rede MLP com duas camadas escondidas. A primeira camada tem 2000 unidades, enquanto a segunda tem 3200 unidades. Para a função de perda é usada a função de entropia cruzada com logist, que é a mais indicada para esse tipo de tarefa. 

In [2]:
def build_deep_fully_network(num_features, num_classes):
    
    # Placeholders
    X_ = tf.placeholder(dtype=tf.float32, shape=[None, num_features])
    Y_ = tf.placeholder(dtype=tf.float32, shape=[None, num_classes])
    
    kernel_initializer = tf.contrib.layers.xavier_initializer()
    
    #camada escondida 1
    layer1 = tf.layers.dense(X_, 2000, activation=tf.nn.relu, kernel_initializer=kernel_initializer)
    
    #camada escondida 2
    layer2 = tf.layers.dense(layer1, 3200, activation=tf.nn.relu, kernel_initializer=kernel_initializer)  
    
    #saída
    logits = tf.layers.dense(layer2, num_classes, activation=None, kernel_initializer=kernel_initializer)
    
    #Loss function
    loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=logits, labels=Y_))

    #Optimizer
    opt = tf.train.AdamOptimizer(learning_rate=0.001).minimize(loss)
    
    output = tf.nn.sigmoid(logits)
    
    #Acc (não usado nessa atividade)
    correct_prediction = tf.equal(tf.round(output), Y_)
    acc = evaluation_step = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

    return opt, loss, acc, output, X_, Y_


## 4 - Iniciando o Tensorflow

O código abaixo inicia uma sessão no TensorFlow. Em seguida, é definida a quantidade de features, 1152 (1024 embeddings visuais + 128 embeddings auditivos). O número de classes de vídeo utilizado é a mesma do YouTube8M, 3862. Por fim, o gráfo de computação é carregado pela chamada da função "build_deep_fully_network".

In [3]:
#Iniciando
sess = tf.InteractiveSession()

num_features = 1152
num_classes = 3862

#construindo o modelo de rede
opt, loss, acc, output, X_, Y_ = build_deep_fully_network(num_features, num_classes)

# inicializando as variveis do tensorflow
sess.run(tf.global_variables_initializer())


## 5 - Treinamento

Nessa etapa o modelo instanciado é treinado com os dados do dataset. Caso a máquina de execução não tenha uma GPU, é possível carregar o modelo pré-treinado ao setar a variavel load_save como True. Caso contrário, o modelo é treinado com 10 épocas. No fim de cada época o estado atual do modelo é avaliado, caso o resultado seja o melhor encontrado, os pesos são salvos.

In [7]:
load_save = True

# restaura pesos salvos, 
saver = tf.train.Saver()

if load_save:
    saver.restore(sess, "save/train_0.508937495467.ckpt")
    
    print("Avaliando o modelo carregado:")
    #test loop com break interno
    while True:
        
        #obtem um batch com: id do batch, vetor de labels e features
        batch_id, labels, features = get_next_val_batch()
        
        #caso o id seja menor que 0, significa que todos os batchs foram usados
        if batch_id < 0:
            break
        
        #alimentando o modelo com os dados e avaliando os resultados 
        feed_dict={X_: features, Y_: labels}
        result = sess.run(output, feed_dict=feed_dict)
        register_batch_evaluation(labels, num_classes, result)
    
    #no termino do loop, os resultados são impressos na tela
    gap, hit1, hit5, hit20 = get_global_evaluation_result()
    print("evaluation: gap", gap, "hit1", hit1, "hit5", hit5, "hit20", hit20)


# caso contrário, inicia o treinamento    
else:
    num_epochs = 10
    best_gap = 0
    for epoch in range(num_epochs):

        #training loop
        while True:
            #obtem um batch com: id do batch, vetor de labels e features
            batch_num, labels, features = get_next_train_batch()
            
            #caso o id seja menor que 0, significa que todos os batchs foram usados
            if batch_num < 0:
                break
            
            feed_dict={X_: features, Y_: labels}
            _, loss = sess.run([opt,loss], feed_dict=feed_dict)

            #print("epoch", epoch,"batch id", batch_num,"loss:", loss)

        #test loop
        while True:
            #obtem um batch com: id do batch, vetor de labels e features
            batch_num, labels, features = get_next_val_batch()
            
            #caso o id seja menor que 0, significa que todos os batchs foram usados
            if batch_num < 0:
                break
            
            #alimentando o modelo com os dados e avaliando os resultados 
            feed_dict={X_: features, Y_: labels}
            result = sess.run(output, feed_dict=feed_dict)
            register_batch_evaluation(labels, num_classes, result)
        
        #no termino do loop, os resultados são impressos na tela
        gap, hit1, hit5, hit20 = get_global_evaluation_result()
        print("validation: epoch", epoch, "- gap", gap, "hit1", hit1, "hit5", hit5, "hit20", hit20)
        
        #apaga os registros da avaliação atual para realizar a avaliacao da próxima época
        clear_registered_evaluations()
        
        #melhor resultado é salvo
        if gap > best_gap:
            best_gap = gap
            save_path = saver.save(sess, "save/train_"+str(best_gap)+".ckpt")
            print("melhor modelo salvo em ...", save_path)
    
        

INFO:tensorflow:Restoring parameters from save/train_0.508937495467.ckpt
Avaliando o modelo carregado:
evaluation: gap 0.5089374954672038 hit1 0.7886026258161525 hit5 0.8854712240016954 hit20 0.9256052395224688


## 6 - Classificando vídeos

Nesta etapa o modelo é usado para classifiacar os vídeos. Nesse experimento usamos apenas as 5 maiores ativações da rede para atribuir até 5 tags ao vídeo. A função "show_top5_labels_info" definida no código abaixo, filtra as top5 labels e exibe suas informações. O parâmetro threshold define um limiar de corte para excluir ativações muito baixas entre as top5 selecionadas.

In [5]:
def show_top5_labels_info(num_classes, output, threshold):
    
    #vetor de features e labels 
    sample_features = output[0]
    labels_id = np.arange(num_classes)
    
    #os vetores são ordenados de forma decrescente pelo valor das ativações
    top5_labels = labels_id[np.argsort(-sample_features)][:5]
    top5_features = sample_features[np.argsort(-sample_features)][:5]
    
    #filtragem das top5 labels
    for index, label in enumerate(top5_labels):
        if top5_features[index] > threshold:
            show_meta_data(label)
        else:
            break
         

O código abaixo seleciona os três primeiros vídeos de cada batch e mostra os metadados das top5 labels.

In [6]:
#cada vez que esse bloco é executado, um batch com novos vídeos é carregado
batch_id, _, features = get_next_train_batch()

video_features1 = np.zeros((1,num_features))
video_features2 = np.zeros((1,num_features))
video_features3 = np.zeros((1,num_features))

#obtendo as features de cada vídeo
video_features1[0] = features[0,:]
video_features2[0] = features[1,:]
video_features3[0] = features[2,:]

result_video1 = sess.run(output, feed_dict={X_: video_features1})
print("batch",batch_id, "video 1 metadata:")
show_top5_labels_info(num_classes, result_video1, 0.)

result_video2 = sess.run(output, feed_dict={X_: video_features2})
print("\n\nbatch",batch_id, "video 2 metadata:")
show_top5_labels_info(num_classes, result_video2, 0.)

result_video3 = sess.run(output, feed_dict={X_: video_features3})
print("\n\nbatch",batch_id, "video 3 metadata:")
show_top5_labels_info(num_classes, result_video3, 0.)


batch 1 video 1 metadata:
- label id: 0 name: Game wiki: https://en.wikipedia.org/wiki/Game
- label id: 23 name: Smartphone wiki: https://en.wikipedia.org/wiki/Smartphone
- label id: 21 name: Mobile phone wiki: https://en.wikipedia.org/wiki/Mobile_phone
- label id: 72 name: Skateboard wiki: https://en.wikipedia.org/wiki/Skateboard
- label id: 5 name: Cartoon wiki: https://en.wikipedia.org/wiki/Cartoon


batch 1 video 2 metadata:
- label id: 62 name: Machine wiki: https://en.wikipedia.org/wiki/Machine
- label id: 1354 name: The Hobbit (film series) wiki: https://en.wikipedia.org/wiki/The_Hobbit_(film_series)
- label id: 8 name: Dance wiki: https://en.wikipedia.org/wiki/Dance
- label id: 14 name: Music video wiki: https://en.wikipedia.org/wiki/Music_video
- label id: 575 name: Forza (series) wiki: https://en.wikipedia.org/wiki/Forza_(series)


batch 1 video 3 metadata:
- label id: 0 name: Game wiki: https://en.wikipedia.org/wiki/Game
- label id: 133 name: Comedy (drama) wiki: https://en.